# Pre-processing Nevada Allocation data for WaDEQA upload.
Date Updated: 08/03/2020
Purpose:  To pre-process the Nevada data into one master file for simple DataFrame creation and extraction

### Notes:
To incldue owners, made a temporary Permit_Owners_5temp.csv file by removing previous onwers from list.  Plan on left-join to POD AllApps_2.csv by 'app' field.


https://ndwr.maps.arcgis.com/home/item.html?id=0d050f7b79724404b80bf29589f67363
https://arcgis.shpo.nv.gov/arcgis/rest/services/Water_Resources_Public_Data/WaterRights_POD_POU/FeatureServer
https://arcgis.shpo.nv.gov/arcgis/rest/services/Water_Resources_Public_Data/SE_HydrographicBasins/FeatureServer
http://water.nv.gov/CodeDefinitions.aspx

In [1]:
#Needed Libararies
import os
import numpy as np
import pandas as pd
from datetime import datetime
pd.set_option('display.max_columns', 999)  # How to display all columns of a Pandas DataFrame in Jupyter Notebook

In [2]:
#Working Directory and Input File
workingDir = "C:/Users/rjame/Documents/WSWC Documents/MappingStatesDataToWaDE2.0/Nevada/WaterAllocation/RawInputData"
os.chdir(workingDir)

OwnTemp = "Permit_Owners_5temp.csv"
PoDAAInput = "POD AllApps_2.csv"

#Dataframe creation
dfown = pd.read_csv(OwnTemp)
dfPoD = pd.read_csv(PoDAAInput)

C:\Users\rjame\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\rjame\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
dfPoD

,OBJECTID,app,cert,app_status,mou,site_name,permit_info,basin,county,source,duty_balance,div_balance,prior_dt,pod_twn,pod_rng,pod_sec,pod_q,pod_qq,latitude,longitude,poly_id,sup_group,x,y
0,1,15809,NaN,DEN,IRR,NaN,http://water.nv.gov/permitinformation.aspx?app...,001,HU,STR,746.67,4.000,9/23/1954 12:00:00 AM,47N,31E,5.0,LT03,,41.988335,-118.527229,NaN,NaN,-118.527229,41.988335
1,2,18001,5536,CER,IRR,001 N47 E30 04B 2,http://water.nv.gov/permitinformation.aspx?app...,001,HU,UG,32.58,0.045,5/14/1959 12:00:00 AM,47N,30E,4.0,TR38,NaN,41.988679,-118.636906,NaN,NaN,-118.636906,41.988679
2,3,19510,NaN,CAN,IRR,NaN,http://water.nv.gov/permitinformation.aspx?app...,001,HU,UG,0.00,0.000,1/30/1961 12:00:00 AM,47N,30E,15.0,NW,NW,41.962780,-118.616112,NaN,NaN,-118.616112,41.962780
3,4,19511,NaN,CAN,IRR,NaN,http://water.nv.gov/permitinformation.aspx?app...,001,HU,UG,0.00,0.000,1/30/1961 12:00:00 AM,47N,30E,15.0,SW,SW,41.951389,-118.616386,NaN,NaN,-118.616386,41.951389
4,5,20574,NaN,CAN,IRR,NaN,http://water.nv.gov/permitinformation.aspx?app...,001,HU,UG,0.00,0.000,7/13/1962 12:00:00 AM,47N,30E,23.0,SW,W2,41.940556,-118.596947,NaN,NaN,-118.596947,41.940556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102946,102947,85073,NaN,PER,IRR,070 N36 E40 32BC 1,http://water.nv.gov/permitinformation.aspx?app...,070,HU,UG,316.80,2.180,1/3/1972 12:00:00 AM,36N,40E,32.0,NW,SW,40.955851,-117.503023,69278.0,2754.0,-117.503023,40.955851
102947,102948,85066,NaN,PER,IRR,070 N36 E39 13CDAD1,http://water.nv.gov/permitinformation.aspx?app...,070,HU,UG,262.98,5.350,6/5/1961 12:00:00 AM,36N,39E,13.0,SW,SE,40.991089,-117.534767,69278.0,2754.0,-117.534767,40.991089
102948,102949,85067,NaN,PER,IRR,070 N36 E40 19BCBB1,http://water.nv.gov/permitinformation.aspx?app...,070,HU,UG,550.00,2.720,6/5/1961 12:00:00 AM,36N,40E,19.0,NW,SW,40.984722,-117.521700,69274.0,2754.0,-117.521700,40.984722
102949,102950,85068,NaN,PER,IRR,070 N36 E40 19CADA1,http://water.nv.gov/permitinformation.aspx?app...,070,HU,UG,913.20,6.000,6/5/1961 12:00:00 AM,36N,40E,19.0,SW,NE,40.979612,-117.515638,69728.0,2754.0,-117.515638,40.979612


In [4]:
dfPoD.columns

Index(['OBJECTID', 'app', 'cert', 'app_status', 'mou', 'site_name',
       'permit_info', 'basin', 'county', 'source', 'duty_balance',
       'div_balance', 'prior_dt', 'pod_twn', 'pod_rng', 'pod_sec', 'pod_q',
       'pod_qq', 'latitude', 'longitude', 'poly_id', 'sup_group', 'x', 'y'],
      dtype='object')

In [5]:
#With owner dataframe, Sort and Merge Columns
dfown = dfown.groupby('app', sort=False).agg(
    lambda x: '; '.join([str(elem) for elem in (list(set(x)))])).reset_index()

In [6]:
#Merge POD and Owner by app field
df = pd.merge(dfPoD, dfown, left_on='app', right_on='app', how='left') # Joinning PoD data

In [7]:
#Changing datatype of used date fields. 
df['prior_dt'] = pd.to_datetime(df['prior_dt'], errors = 'coerce')
df['prior_dt'] = pd.to_datetime(df["prior_dt"].dt.strftime('%m/%d/%Y'))

In [8]:
#Removing empty prior_dt fields.
dropIndex = df.loc[(df['prior_dt'].isnull()) | (df['prior_dt'] == '')].index

if len(dropIndex) > 0:
    df = df.drop(dropIndex)
    df = df.reset_index(drop=True)

In [9]:
df

,OBJECTID_x,app,cert,app_status,mou,site_name,permit_info,basin,county_x,source,duty_balance,div_balance,prior_dt,pod_twn,pod_rng,pod_sec,pod_q,pod_qq,latitude,longitude,poly_id,sup_group,x,y,OBJECTID_y,owner_name,owner_type,owner_pct,owner_duty,owner_div_rate,owner_acre,owner_chg_app,county_y,apn,owner_remark
0,1,15809,NaN,DEN,IRR,NaN,http://water.nv.gov/permitinformation.aspx?app...,001,HU,STR,746.67,4.000,1954-09-23,47N,31E,5.0,LT03,,41.988335,-118.527229,NaN,NaN,-118.527229,41.988335,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,18001,5536,CER,IRR,001 N47 E30 04B 2,http://water.nv.gov/permitinformation.aspx?app...,001,HU,UG,32.58,0.045,1959-05-14,47N,30E,4.0,TR38,NaN,41.988679,-118.636906,NaN,NaN,-118.636906,41.988679,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,19510,NaN,CAN,IRR,NaN,http://water.nv.gov/permitinformation.aspx?app...,001,HU,UG,0.00,0.000,1961-01-30,47N,30E,15.0,NW,NW,41.962780,-118.616112,NaN,NaN,-118.616112,41.962780,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,19511,NaN,CAN,IRR,NaN,http://water.nv.gov/permitinformation.aspx?app...,001,HU,UG,0.00,0.000,1961-01-30,47N,30E,15.0,SW,SW,41.951389,-118.616386,NaN,NaN,-118.616386,41.951389,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,20574,NaN,CAN,IRR,NaN,http://water.nv.gov/permitinformation.aspx?app...,001,HU,UG,0.00,0.000,1962-07-13,47N,30E,23.0,SW,W2,41.940556,-118.596947,NaN,NaN,-118.596947,41.940556,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101105,102947,85073,NaN,PER,IRR,070 N36 E40 32BC 1,http://water.nv.gov/permitinformation.aspx?app...,070,HU,UG,316.80,2.180,1972-01-03,36N,40E,32.0,NW,SW,40.955851,-117.503023,69278.0,2754.0,-117.503023,40.955851,169709,BUTTONPOINT LIMITED PARTNERSHIP,C,0.0,0.0,0.0,0.0,,,nan,nan
101106,102948,85066,NaN,PER,IRR,070 N36 E39 13CDAD1,http://water.nv.gov/permitinformation.aspx?app...,070,HU,UG,262.98,5.350,1961-06-05,36N,39E,13.0,SW,SE,40.991089,-117.534767,69278.0,2754.0,-117.534767,40.991089,169694,BUTTONPOINT LIMITED PARTNERSHIP,C,0.0,0.0,0.0,0.0,,,nan,nan
101107,102949,85067,NaN,PER,IRR,070 N36 E40 19BCBB1,http://water.nv.gov/permitinformation.aspx?app...,070,HU,UG,550.00,2.720,1961-06-05,36N,40E,19.0,NW,SW,40.984722,-117.521700,69274.0,2754.0,-117.521700,40.984722,169696,BUTTONPOINT LIMITED PARTNERSHIP,C,0.0,0.0,0.0,0.0,,,nan,nan
101108,102950,85068,NaN,PER,IRR,070 N36 E40 19CADA1,http://water.nv.gov/permitinformation.aspx?app...,070,HU,UG,913.20,6.000,1961-06-05,36N,40E,19.0,SW,NE,40.979612,-117.515638,69728.0,2754.0,-117.515638,40.979612,169698,BUTTONPOINT LIMITED PARTNERSHIP,C,0.0,0.0,0.0,0.0,,,nan,nan


In [10]:
#Exporting to Finished File
df.to_csv('P_MastersNV.csv', index=False)  # The output